In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
import pandas as pd
sns.set()
import numpy as np # linear algebra
 # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  *
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.applications import DenseNet121, VGG19, ResNet50
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import PIL.Image
import matplotlib.pyplot as mpimg
import os
import ssl
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.preprocessing import image

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from sklearn.utils import shuffle

#    CL1 Verinin hazırlanması
#    CL2 ANN Modelinin hazırlanması, ANN Eğitilmesi, Tahmin
#    CL3 Sonuçlar

In [ ]:
class DataPrep(object):
    total_data = None
    
    def __init__(self):
        self.covid_for_test = None
        self.train_data = None
        self.test_data = None
        self.merged_test_data = None
        self.final_test_data = None
        self.final_train_data = None
        self.y_train = None
        self.train_arrays = None
        self.test_arrays = None
        self.corona_augmented = None
    
    def getData(self):
        input_df = pd.read_csv('../input/coronahack-chest-xraydataset/Chest_xray_Corona_Metadata.csv')
        input_ratio = 0.725
        self.total_data = input_df.shape[0]
        train_data = input_df[input_df['Dataset_type'] == 'TRAIN']
        train_data = train_data.sample(int(train_data.shape[0]*input_ratio))
        test_data = input_df[input_df['Dataset_type'] == 'TEST']
        test_data = test_data.sample(int(test_data.shape[0]*input_ratio))
        #assert train_data.shape[0] + test_data.shape[0] == input_df.shape[0]
        print(f"Shape of train data : {train_data.shape}")
        print(f"Shape of test data : {test_data.shape}")
        moving_ratio = 0.17
        
        covid_path = train_data[train_data['Label_2_Virus_category']=='COVID-19'] #['X_ray_image_name'].values     
        #covid_path = pd.DataFrame(covid_path)
        print("covid_path shape: ", covid_path.shape)
        covid_for_test = covid_path.sample(int(round(len(covid_path)*moving_ratio,0)))
        print("covid_for_test shape:",covid_for_test.shape)
        print(type(covid_for_test))
        def flatten(l):
            try:
                return flatten(l[0]) + (flatten(l[1:]) if len(l) > 1 else []) if type(l) is list else [l]
            except IndexError:
                return []
        #covid_for_test = covid_for_test.values.tolist()
        #covid_for_test = flatten(covid_for_test)
        #print(covid_for_test)
        
        print(len(train_data))
        self.train_data = train_data[~train_data['X_ray_image_name'].isin(covid_for_test['X_ray_image_name'].values)]
        #train_data.drop(train_data[train_data["X_ray_image_name"] in buffer], inplace = True)
        print("shape traindata",self.train_data.shape)
        
        # Defining the path to Train and Test directories
        #training_data_path = '../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train'
        #testing_data_path = '../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test'
        self.covid_for_test = covid_for_test
        self.test_data = test_data
        
        
    def classifyData(self):
        test_img_dir = '/kaggle/input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test'
        train_img_dir = '/kaggle/input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train'
        #covid_for_test_full_path = list(map(lambda x: os.path.join(train_img_dir, x), self.covid_for_test))
        #print(covid_for_test_full_path)
        self.merged_test_data = self.test_data.append(self.covid_for_test)
        print("self.merged_test_data shape:", self.test_data.shape)
        
        # add a target and class feature
        self.train_data['class'] = self.train_data.Label_2_Virus_category.apply(lambda x: 1 if x=='COVID-19' else 0)
        print(self.train_data)
        self.merged_test_data['class'] = self.merged_test_data.Label_2_Virus_category.apply(lambda x: 1 if x=='COVID-19' else 0)
        #print("birleştirilmiş test setindeki covidliler: ", self.test_data[['Label_2_Virus_category']=='COVID-19'])
        print(self.merged_test_data[self.merged_test_data['Label_2_Virus_category']=='COVID-19'])

        #get the important features
        self.final_train_data = self.train_data[['X_ray_image_name', 'class']]
        self.final_test_data = self.merged_test_data[['X_ray_image_name', 'class']]
        
        print("shape of self.final_test_data:", self.final_test_data.shape)
        print(self.final_train_data.shape)
    def visualization(self):
        pass
    
    def augmentation(self):
        #create a imagegenerator for for augmentation
        datagen =  ImageDataGenerator(shear_range=0.2, zoom_range=0.2)
        train_img_dir = '/kaggle/input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train'
        test_img_dir = '/kaggle/input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test'    
        def read_img(filename, size, path):
            img = keras.preprocessing.image.load_img(os.path.join(path, filename), target_size=size)
            
            #convert image to array
            img = img_to_array(img) / 255
            return img
        #augment the images labeled with covid-19 to balance the data
        corona_df = self.final_train_data[self.final_train_data['class'] == 1 ]
        self.corona_augmented = []

        #create a function for augmentation
        def augment(name):
            img = read_img(name, (255,255), train_img_dir)
            i = 0
            for batch in tqdm(datagen.flow(tf.expand_dims(img, 0), batch_size=32)):
                self.corona_augmented.append(tf.squeeze(batch).numpy())
                if i == 20:
                    break
                i =i+1

        #apply the augment function       
        corona_df['X_ray_image_name'].apply(augment)
        print("Türetilen veri:", len(self.corona_augmented))
        
        # extract the image from training data and test data, then convert them as array
        self.train_arrays = []
        self.final_train_data['X_ray_image_name'].apply(lambda x: self.train_arrays.append(read_img(x, (255,255), train_img_dir)))
        
        self.test_arrays = []
        self.test_data['X_ray_image_name'].apply(lambda x: self.test_arrays.append(read_img(x, (255,255), test_img_dir)))
        self.covid_for_test['X_ray_image_name'].apply(lambda x: self.test_arrays.append(read_img(x, (255,255), train_img_dir)))
        
        print("train_arrays length:",len(self.train_arrays))  
        print("test_arrays length:",len(self.test_arrays))  
        print("shape:", self.train_arrays[0].shape)
        
        #concatenate the training data labels and the labels for augmented images
        self.y_train = np.concatenate((np.int64(self.final_train_data['class'].values), np.ones(len(self.corona_augmented), dtype=np.int64)))
    
if __name__ == "__main__":
    prep = DataPrep()
    prep.getData()
    prep.classifyData()
    prep.augmentation()
    #print(prep.total_data)

In [ ]:
class Net(object):
    def __init__(self, train_arrays, test_arrays, y_train,
                final_test_data, corona_augmented):
        self.train_tensors = tf.convert_to_tensor(np.concatenate((np.array(train_arrays), np.array(corona_augmented))))
        self.test_tensors  = tf.convert_to_tensor(np.array(test_arrays))
        
        self.y_train_tensor = tf.convert_to_tensor(y_train)
        self.y_test_tensor = tf.convert_to_tensor(final_test_data['class'].values)
        
        train_dataset = tf.data.Dataset.from_tensor_slices((self.train_tensors, self.y_train_tensor))
        test_dataset = tf.data.Dataset.from_tensor_slices((self.test_tensors, self.y_test_tensor))
        
        batch_size = 16
        sh_buffer = 1773

        self.train_batches = train_dataset.shuffle(sh_buffer).batch(batch_size)
        self.test_batches = test_dataset.batch(batch_size)
        
    def start(self): 
        ssl._create_default_https_context = ssl._create_unverified_context
        #define input shape
        inp_shape = (255,255,3)
        #get the pretrained model
        
        base_model = tf.keras.applications.ResNet50(input_shape = inp_shape,
                                            include_top = False,
                                            weights='imagenet')
        base_model.trainable = False
        
        # Construct the model
        model = Sequential()
        model.add(base_model)
        model.add(GlobalAveragePooling2D())
        model.add(Dense(128))
        model.add(Dropout(0.2))
        model.add(Dense(1, activation = 'sigmoid'))
        
        #add a earlystopping callback to stop the training if the model is not learning anymore
        callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
        
        # Define optimizer, loss function and performance metric for the training.
        model.compile(optimizer='adam',
              loss = 'binary_crossentropy',
              metrics=['accuracy'])
        # Train the model
        model.fit(self.train_batches, epochs=10, validation_data=self.test_batches, callbacks=[callbacks])
        

In [ ]:
net = Net(prep.train_arrays, prep.test_arrays, prep.y_train, prep.final_test_data, prep.corona_augmented)
prep = None
net.start()



In [ ]:
#print(len(net.train_tensors))

In [ ]:
class Training(object):
    pass
    

In [ ]:
class Results(object):
    pass
    